In [1]:
# %load_ext autoreload
# %autoreload 2

# Tip for quick search

* Needs attention: the place where needs update or better logic
* question to be answered: the place where things are still not clear
* Unit Test: Unit test where you can drill in to find the data that leads to the check results for a specific project and specific check
* TODO: things needs to be done
* bookmark: stop point from last visit


# Update Note: 



# Admin Notes:


1. The AMTool dataset is archived daily as csv files and used for the project book check. 
The csv files are located at: 
r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\DataLake'

2. The excel input files are checked daily and archived with datestamp whenever it is modified.
The continuously updated excel input files are located at: r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\excel'
The excel input file are archived at: r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Tableau Dashboards\Data_MiscInput'
To recover the archived excel file used in project book check for a target date, select the excel file with latest datestamp but is still earlier than the target date.

3. The check summary export action is logged daily. It can be used for daily monitoring. 
The file export log is located at: \\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal\log

4. The published data are at:

    * csv files for district asset manager: http://svgcshopp.dot.ca.gov/DataLake/ProjectBookCheck/
    * csv files for HQ AM: \\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_WorkingFolder\output_internal
    * tableau workbook with live data source: https://tableau.dot.ca.gov/#/site/AssetManagement/workbooks/1815/views


<a id='TableOfContents'></a>

# Table Of Contents

## Data Preprocessing

### [Global Constants](#GlobalConstants)


### [Load and cleanup source data](#Read_Data)

* [Bridge_Inventory](#Bridge_Inventory)
* [Bridge_Worksheet](#Bridge_Worksheet)
* [Check_Exceptions](#Check_Exceptions)
* [Counties](#Counties)
* [Drainage_Worksheet](#Drainage_Worksheet)
* [Minor_Raw_Data](#Minor_Raw_Data)
* [Pavement_Worksheet](#Pavement_Worksheet)
* [PID_Workload](#PID_Workload)
* [Postmile_Check](#Postmile_Check)
* [Programming_Summary](#Programming_Summary)
* [ProgrammingList](#ProgrammingList)
* [Project_Detail_Report](#Project_Detail_Report)
* [Project_Obselete](#Project_Obselete)
* [SHOPP_Candidates](#SHOPP_Candidates)
* [SHOPP_Raw_Data](#SHOPP_Raw_Data)
* [TenYrShopp_Perf_RawData](#TenYrShopp_Perf_RawData)
* [TMS_Worksheet](#TMS_Worksheet)


## Add fields to SHOPP raw data (calculate and join)
* [Calculated Fields](#AddDataColumns)
* [Join Tables](#DataJoining)



## Data Check and Export


## [Data Check List](#Issue_Table1)
The main table of check issues, 
one issue per row, 



## [New Checks](#NewChecks)
* 'Check Safety Comment'
#Check comments about ActID of E55 and E58
#if ActID == E55 and E58 and the comments include: 'HQ added the activity and District needs to review it.', mark the project
* 'TMS data update needed for the project?'

## [Export repeated assets](#Export_repeated_assets)
* repeated assets (csv, tableau datasource)




## [Final Clean Up](#FinalCleanUp)


# Import common modules

In [2]:

from datetime import datetime
import os.path

# import requests
import pandas as pd

import numpy as np
import re

import shutil

In [3]:
import time
start_time = time.time()

In [4]:
#show dataframe without skip column
pd.options.display.max_columns = 100

In [5]:
# from config_datasource import *
# from projectbookcheck_utilityfunction import *
from constants import *
import projectbookcheck_utilityfunction as uf

You are using the Extract API 2.0, please save the output as .hyper format


# General Approach

use SHOPP raw data as basis for data checks. 
Each project only occupies one line

can expand columns, only if it will not create duplicate rows in the SHOPP raw dataset. 


Question: what is the entire list of projects, how to include projects without AMT_ID (SHOPP ID)
Ans: use raw data, check missing SHOPP ID seperately 


# Data update procedure

data schema: name, data type, default value if missing

Option 1: Keep Excel column header fixed (number and sequence)

Option 2: Maintain a dictionary of Excel column name and dataframe column name



# Check update procedure

Create utility function in python, in seperate module
add the additional check in the main ETL module
update the final data visualization


# Data clean process

* funding amount: remove dollar sign, 
* fill missing value, string, numerical, 
* remove leading single quote for string value
* strip off leading and trailing space 

* regulate column names




<a id='GlobalConstants'></a>
## Global Constants

In [6]:
CURRENT_FY = uf.fiscalyear(datetime.today())

In [7]:

#override to get the live data
# DATA_SOURCE_TYPE = 'live'


# %load_ext autoreload
# %autoreload 2


# PROJECTBOOKCHECK_INPUT_FOLDER = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_DataChecksSupport\dev\input'
# PROJECTBOOKCHECK_HTTPSERVER_FOLDER = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_DataChecksSupport\dev\output'
# PROJECTBOOKCHECK_OUTPUT_FOLDER = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_DataChecksSupport\dev\output'

# LOG_FILE = r'\\ct.dot.ca.gov\dfshq\DIROFC\Asset Management\4e Project Book\Projectbook_DataChecksSupport\dev\log\ProjectBookExport.log'


<a id='Read_Data'></a>

# Read Data


In [8]:
File_TimeStamp = '' #always read the latest timestamped file

if DATA_SOURCE_TYPE == 'csv':
    filename = 'TenYrShopp_RawData_'
    df_SHOPP_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp))

    filename = 'TenYrShopp_PerfM_Raw_Data_'
    df_perf_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp))

    filename = 'Rawdata_Bridge_Worksheet_'
    df_brg_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), skiprows = [0], header = 0)

    filename = 'Rawdata_Pavement_Worksheet_'
    df_pav_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), skiprows = [0], header = 1)

    filename = 'Rawdata_Drainage_Worksheet_'
    df_drain_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), header = 0)

    filename = 'Rawdata_TMS_Worksheet_'
    df_tms_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), header = 0)

    filename = 'Programming_Summary_'
    df_program_summary = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), header = 0)

    filename = 'Minor_Project_Details_Raw_Data_'
    df_Minor_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp))

    filename = 'HM_Project_Details_Raw_Data_'
    df_HM_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp))

    filename = 'Minor_Rawdata_TMS_Worksheet_'
    df_Minor_tms_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), header = 0) 

    filename = 'HM_Rawdata_TMS_Worksheet_'
    df_HM_tms_raw_data = pd.read_csv(r'{}\{}{}.csv'.format(DATALAKE_FOLDER, filename, File_TimeStamp), header = 0) 
    
    path_to_file = r'{}\{}.csv'.format(DATALAKE_FOLDER, filename)
    t = os.path.getmtime(path_to_file)
    Data_TimeStamp = datetime.fromtimestamp(t).strftime("%m-%d-%Y %H:%M:%S")
    TARGETDATE = datetime.fromtimestamp(t).strftime("%m-%d-%Y")
    
else:
    print('skip getting csv data.')

C:\Users\s151589\Anaconda3\envs\dataprep\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\s151589\Anaconda3\envs\dataprep\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (29,30,31,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
filename = 'TMS_Life_Cycle.xlsx'

df_TMS_LifeCycle = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename)) 

In [10]:
filename = 'Project Detail Report.csv'

df_Project_Details = pd.read_csv(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename)) 

dict_rename = {
    'Project ID': 'EFIS',
    }
df_Project_Details= df_Project_Details.rename(dict_rename, axis = 1)

df_Project_Details['EFIS'] = pd.to_numeric(df_Project_Details['EFIS'], errors='coerce')

In [11]:
filename = 'Minor Approved list.xlsx'

df_Minor_ApprovedList = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename)) 

dict_rename = {
    'Project ID': 'EFIS',
    }
df_Minor_ApprovedList = df_Minor_ApprovedList.rename(dict_rename, axis = 1)

<a id='Minor_Raw_Data'></a>

## Minor and HM Raw Data


In [12]:

df_Minor_raw_data['District'] = df_Minor_raw_data['District'].apply(uf.remove_punction)
df_Minor_raw_data['District'] = df_Minor_raw_data['District'].astype(int)

df_Minor_raw_data['Unique EA'] = df_Minor_raw_data.apply(uf.calc_unique_EA, axis = 1)

dict_rename = {'ID': 'AMT_ID',
    'Project ID':'EFIS',
    'FY.2': 'RTL'    
              }
df_Minor_raw_data = df_Minor_raw_data.rename(dict_rename, axis = 1)

#conver EFIS to numeric, 
#filter null and 0 


df_Minor_raw_data['EFIS'] = pd.to_numeric(df_Minor_raw_data['EFIS'], errors='coerce')
df_Minor_raw_data = df_Minor_raw_data[(df_Minor_raw_data['EFIS'].notna()) & df_Minor_raw_data['EFIS'] != 0]



In [13]:
df_HM_raw_data['Unique EA'] = df_HM_raw_data.apply(uf.calc_unique_EA, axis = 1)

df_HM_raw_data['EFIS'] = pd.to_numeric(df_HM_raw_data['EFIS'], errors='coerce')
df_HM_raw_data = df_HM_raw_data[(df_HM_raw_data['EFIS'].notna()) & df_HM_raw_data['EFIS'] != 0]

#rename ID
dict_rename = {'ID': 'AMT_ID',
              'Award FY.2': 'RTL'
              }
df_HM_raw_data= df_HM_raw_data.rename(dict_rename, axis = 1)

<a id='SHOPP_Raw_Data'></a>
## SHOPP Raw Data

In [14]:
df_SHOPP_raw_data.name = 'df_SHOPP_raw_data'

In [15]:
#rename columns 

dict_rename_rawdata = {
                       'County':'County TYP',
                       'Route': 'Route TYP',
                       'BackPM':'BackPM TYP',
                       'AheadPM':'AheadPM TYP',
                       'ID': 'AMT_ID',
                       'Ten-Year Plan': 'Ten-Year Plan RD',
                       'County.1' : 'County PRG',
                       'Route.1' : 'Route PRG',
                       'BackPM.1':'BackPM PRG',
                       'AheadPM.1' : 'AheadPM PRG',
                       'County.2' : 'County PCR',
                       'Route.2' : 'Route PCR',
                       'BackPM.2':'BackPM PCR',
                       'AheadPM.2' : 'AheadPM PCR',
                       'Activity Category': 'Activity',
                       'RTL In Use': 'RTL'
                      }
df_SHOPP_raw_data = df_SHOPP_raw_data.rename(dict_rename_rawdata, axis = 1)



In [16]:
#remove leading puncture for target columns
cols_strip = ['District','Route TYP','EA','EFIS','Route PRG','PPNO','Route PCR']
for c in cols_strip :
    df_SHOPP_raw_data[c] = df_SHOPP_raw_data[c].str.strip("'")


In [17]:

cost_columns = [
    'RW Cost ($K)',
    'Const Cost ($K)',
    'Support Cost ($)',
    'TYP Total Project Cost ($K)',
    'PAED ($K)',
    'PSE ($K)',
    'R/W ($K)',
    'CONS ($K)',
    'Prog Support Cost ($)',
    'Prog RW Cost ($K)',
    'Prog Const Cost ($K)',
    'Prog Total Project Cost ($K)',
    'PCR R/W Cap ($K)',
    'PCR Const Cap ($K)',
    'PCR Support Cost ($K)',
    'PCR Total Cost ($K)',
    'Project Cost In Use',
    'Total LL Prog ($K)',
    'LL PAED Cost ($K)',
    'LL CONS Cap ($K)'
               ]

for c in cost_columns:
    df_SHOPP_raw_data[c] = df_SHOPP_raw_data[c].apply(uf.curreny_to_float)
    df_SHOPP_raw_data[c].fillna(0, inplace = True)


In [18]:

#data clean 
#data type regulation
#string/text data regulation

df_SHOPP_raw_data['District'] =df_SHOPP_raw_data['District'].astype(int)

df_SHOPP_raw_data['EFIS'] = pd.to_numeric(df_SHOPP_raw_data['EFIS'], errors='coerce')

df_SHOPP_raw_data['Route PCR'] = df_SHOPP_raw_data['Route PCR'].astype(str)

#data trimming
#row trimming
df_SHOPP_raw_data= df_SHOPP_raw_data[df_SHOPP_raw_data['District'] != 56]

#column trimming
df_SHOPP_raw_data.drop(['District Priority', 'PIR Performance Report'],
  axis='columns', inplace=True, errors='ignore')



#Question to be answered:
#for EA Raw Data, the missing data is not null , but ''. Should we handle the missing data uniformly for string data?

#TODO:
#fill missing data
#data quality check (checksum,)

In [19]:
df_SHOPP_raw_data['CCA Date Miilestone (M600)'] = df_SHOPP_raw_data['CCA Date Miilestone (M600)'].apply(uf.regulate_timestamp_format)

In [20]:
df_SHOPP_raw_data['Section'] = df_SHOPP_raw_data['Section In Use']

<a id='TenYrShopp_Perf_RawData'></a>
## TenYrShopp_Perf_RawData


In [21]:
#rename columns
dict_rename_perf_rawdata = {
                           'ID': 'AMT_ID',
#                             'ProjectedRTL FY': 'Projected RTL FY',
    
# 'ActID':'Performance_ActID',
# 'Quantity':    'Performance_Quantity'
              }

df_perf_raw_data = df_perf_raw_data.rename(dict_rename_perf_rawdata, axis = 1)

In [22]:
cols_strip = ['EA','EFIS','PPNO']
for c in cols_strip :
    df_perf_raw_data[c] = df_perf_raw_data[c].str.strip("'")

In [23]:
# df_perf_raw_data.columns

In [24]:
#data clean 
#data type regulation

df_perf_raw_data['Quantity'] = df_perf_raw_data['Quantity'].fillna(0)
df_perf_raw_data['Assets in Good Cond'] = df_perf_raw_data['Assets in Good Cond'].fillna(0)
df_perf_raw_data['Assets in Fair Cond'] = df_perf_raw_data['Assets in Fair Cond'].fillna(0)
df_perf_raw_data['Assets in Poor Cond'] = df_perf_raw_data['Assets in Poor Cond'].fillna(0)
df_perf_raw_data['New Assets Added'] = df_perf_raw_data['New Assets Added'].fillna(0)

df_perf_raw_data['EFIS'] = pd.to_numeric(df_perf_raw_data['EFIS'], errors='coerce')


In [25]:
#data trimming
#row
df_perf_raw_data= df_perf_raw_data[df_perf_raw_data['District'] != 56]
#column
df_perf_raw_data.drop(['PID Cycle', 'TYP','ProjectedSHOPP Cycle','RequestedRTL FY','DistrictPriority'],
  axis='columns', inplace=True, errors='ignore')

In [26]:
df_perf_raw_data.name = 'df_perf_raw_data'

<a id='ProgrammingList'></a>
## Programming List


In [27]:
shts = ['2010 SHOPP',
        '2012 SHOPP',
        '2014 SHOPP',
        '2016 SHOPP',
        '2018 SHOPP',
        '2020 SHOPP',
        'Long Lead'
        ]

filename = 'Programming_list.xlsx'

df_dict = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename), sheet_name =shts) 


df_program = pd.DataFrame()

for k, v in df_dict.items():
#     print(type(v))
#     print(k)
    v['Table Names'] = k
#     print(v.columns)
    df_program = df_program.append(v)

In [28]:
#rename columns
dict_rename_program = {
#                         'EA':'EA', 
                        'EFIS':'EFIS_Program', 
#                         'PPNO':'PPNO Programming', 
                        'Total Capital & Support':'Total Capital & Support Cost',
#                         'Route': 'Route Programming',
              }

df_program = df_program.rename(dict_rename_program, axis = 1)

In [29]:
#data clean 
#data type regulation
# df_program['Dist'] = df_program['Dist'].astype(int)
# df_program['EFIS'] = pd.to_numeric(df_program['EFIS'], errors='coerce')

fillna_columns = ['Con Sup','RW Sup','PA&ED','PS&E', 'PA&ED', 'RW', 'Con']

df_program[fillna_columns].fillna(0, inplace=True)
df_program['Route'].fillna('Various', inplace=True)

df_program['Support Cost'] = df_program['Con Sup']+df_program['RW Sup']+df_program['PA&ED']+df_program['PS&E']
df_program['Capital Cost'] = df_program['Con']+df_program['RW']


C:\Users\s151589\Anaconda3\envs\dataprep\lib\site-packages\pandas\core\frame.py:4459: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [30]:
# df_program[['Con Sup','RW Sup','PA&ED','PS&E']].info()

In [31]:
# df_program['PA&ED'].value_counts()

In [32]:
#remove leading puncture for all string columns
# df_program.head()

In [33]:
#data augmentation
#data transformation

# df_program.dropna(subset = ['EFIS_Program'], inplace = True)
df_program['FY'] = df_program['FY'].apply(uf.FY_cleanup)

df_program['Begin Post Miles'] = df_program['Post Miles'].apply(lambda x: str(x).split('/')[0])
df_program['End Post Miles'] = df_program['Post Miles'].apply(lambda x: str(x).split('/')[0] if '/' in str(x) else np.NaN)

In [34]:
#data trimming
df_program.drop(['PM1BF', 'PM1B', 'PM1AF','PM1A',], axis='columns', inplace=True)

In [35]:
df_program.name = 'df_program'

In [36]:
# df_program.head()

In [37]:
# df_program.columns

In [38]:
#data sanity check
#check duplicates
#check null
#check data type

if df_program['EFIS_Program'].value_counts(dropna = False).max() > 1:
    Print('Duplicate EFIS ID found, please check the source data')

In [39]:
df_program['EFIS_Program'].isna().sum()

0

<a id='Bridge_Inventory'></a>

## Bridge Inventory

In [40]:
filename = 'GFP_BrInvList_AllDistricts.xlsx'

df_bridge_inventory = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename), skiprows = 4, header = [0]) 

In [41]:

# filename = 'GFP_BrInvList_AllDistricts_March_2020_05042020.xlsx'

# df_bridge_inventory = pd.read_excel(r'{}\{}'.format(PROJECTBOOKCHECK_INPUT_FOLDER, filename), skiprows = 4, header = [0]) 

In [42]:
keep_columns = ['Bridge #', 'Deck Area, SF', 'Date', 'Health', 'Deck (58)',
       'Super (59)', 'Sub (60)', 'Culv (62)', 'Scour', 'Seismic', 'Overall',
       'VC', 'Permit', 'Rail Overall', 'Good, LF', 'Fair, LF', 'Poor, LF',
       'Category']

In [43]:
df_bridge_inventory = df_bridge_inventory[keep_columns]

In [44]:
#rename columns 

dict_rename_bridge_inventory = {'Bridge #':'BridgeNo',
                               'Date':'Inspection Date', 
                               'Health': 'Bridge Health',
                                'Scour':'Bridge Scour', 
                                'Seismic':'Bridge Seismic', 
                               'Deck (58)' : 'NBI Condition Ratings_Deck (58)',
                                'Super (59)': 'NBI Condition Ratings_Super (59)',
                                'Sub (60)':'NBI Condition Ratings_Sub (60)',
                                'Culv (62)':'NBI Condition Ratings_Culv (62)',
                                'Overall': 'Bridge Goods Movement_Overall',
                                'VC': 'Bridge Goods Movement_VC',
                                'Permit':'Bridge Goods Movement_Permit',
                                'Rail Overall': 'Bridge Rail Upgrade_Rail Overall', 
                                'Good, LF': 'Bridge Rail Upgrade_Good, LF', 
                                'Fair, LF': 'Bridge Rail Upgrade_Fair, LF', 
                                'Poor, LF': 'Bridge Rail Upgrade_Poor, LF', 
                      }

df_bridge_inventory.rename(dict_rename_bridge_inventory, axis = 1, inplace = True)

In [45]:
def default_to_good(value):
    if value and value in ['Poor','Fair','Good']:
        return value
    else:
        return 'Good'

data_recondition_columns = ['Bridge Health','Bridge Scour','Bridge Seismic','Bridge Goods Movement_Overall',
       'Bridge Goods Movement_VC', 'Bridge Goods Movement_Permit','Bridge Rail Upgrade_Rail Overall',]

for c in data_recondition_columns:
    df_bridge_inventory[c] = df_bridge_inventory[c].apply(default_to_good)

In [46]:
df_bridge_inventory.name = 'df_bridge_inventory'

In [47]:
# df_bridge_inventory.shape

<a id='Bridge_Worksheet'></a>

## Raw Data Bridge Worksheet


In [48]:
#rename columns 
#with manual edits

dict_rename_bridge_worksheet = {
 'ID': 'AMT_ID',
 'Bridge №': 'BridgeNo',
 'Work Type': 'WorkType',
 'Brdige / TunnelWork Description': 'WorkDescription',
 'Bridge /TunnelHealth Pre': 'Health Pre',
 'Bridge /TunnelHealth Post': 'Health Post',
 'BridgeScourPre': 'Scour_Pre',
 'BridgeScourPost': 'Scour_Post',
 'BridgeSeismicPre': 'Seismic_Pre',
 'BridgeSeismicPost': 'Seismic_Post',
 'BridgeGds MvmtPre': 'GdsMvmt_Pre',
 'BridgeGds MvmtPost': 'GdsMvmt_Post',
 'Exist(sf)': 'Deck_Exist(sf)',
 'Additional(sf)': 'Deck_Additional(sf)',
 'Y/N': 'Paint_Y/N',
 'Condition': 'Paint_Condition',
 'Paint Area(sf)': 'Paint Area(sf)',
 'Y/N.1': 'ElectricalMechanical_Y/N',
 'Condition.1': 'ElectricalMechanical_Condition',
 'Area(sf)': 'ElectricalMechanical_Area(sf)',
 'Y/N.2': 'ApproachSlab_Y/N',
 'Replaced(sf)': 'ApproachSlab_Replaced(sf)',
 'New(sf)': 'ApproachSlab_New(sf)',
 'Y/N.3': 'Rail_Y/N',
 'Good(lf)': 'Rail_Good(lf)',
 'Fair(lf)': 'Rail_Fair(lf)',
 'Poor(lf)': 'Rail_Poor(lf)',
 'Additonal(lf)': 'Rail_Additonal(lf)',
 'Post Good(lf)': 'Rail_Post Good(lf)',
 'Post Fair(lf)': 'Rail_Post Fair(lf)',
 'Post Poor(lf)': 'Rail_Post Poor(lf)',
 'Post New(lf)': 'Rail_Post New(lf)',
 'FishPassage(Y/N)': 'FishPassage(Y/N)',
}

df_brg_raw_data.rename(dict_rename_bridge_worksheet, axis = 1, inplace = True)

In [49]:
df_brg_raw_data.name = 'df_brg_raw_data'

In [50]:
df_brg_raw_data['Rail_Good(lf)'].fillna(0, inplace = True)
df_brg_raw_data['Rail_Fair(lf)'].fillna(0, inplace = True)
df_brg_raw_data['Rail_Poor(lf)'].fillna(0, inplace = True)

In [51]:
df_brg_raw_data['Rail_Total(lf)'] = (df_brg_raw_data['Rail_Good(lf)'] 
                                             + df_brg_raw_data[ 'Rail_Fair(lf)'] 
                                             + df_brg_raw_data['Rail_Poor(lf)'])

<a id='Pavement_Worksheet'></a>

## Raw Data Pavement Worksheet


In [52]:
#rename columns 
#with manual editing

dict_rename_pavement_worksheet = {
 'ID': 'AMT_ID',
 'Class': 'RoadwayClass',
 'Green': 'TriditionalCondition_Green',
 'Yellow': 'TriditionalCondition_Yellow',
 'Blue': 'TriditionalCondition_Blue',
 'Orange': 'TriditionalCondition_Orange',
 'Red': 'TriditionalCondition_Red',
 'Good': 'MAP21_Good',
 'Fair': 'MAP21_Fair',
 'Poor': 'MAP21_Poor',
 'Total LaneMiles': 'Total LaneMiles',
 'Green.1': 'TriditionalCondition_Post_Green',
 'Yellow.1': 'TriditionalCondition_Post_Yellow',
 'Blue.1': 'TriditionalCondition_Post_Blue',
 'Orange.1': 'TriditionalCondition_Post_Orange',
 'Red.1': 'TriditionalCondition_Post_Red',
 'Good.1': 'MAP21_Post_Good',
 'Fair.1': 'MAP21_Post_Fair',
 'Poor.1': 'MAP21_Post_Poor',
#  'Date': 'Date'
                               }
df_pav_raw_data.rename(dict_rename_pavement_worksheet, axis = 1, inplace = True)

In [53]:
df_pav_raw_data.name = 'df_pav_raw_data'

In [54]:
# df_pav_raw_data.shape

In [55]:
# df_pav_raw_data['Plan Year']

<a id='Drainage_Worksheet'></a>
## Raw Data Drainage Worksheet


In [56]:
df_drain_raw_data.name = 'df_drain_raw_data'

dict_drain_rename = {
 'ID': 'AMT_ID',
 'Data Date':'Data Date_Drainage'
                               }
df_drain_raw_data.rename(dict_drain_rename, axis = 1, inplace = True)

cols = ['EA','EFIS','SYSNO','INETNO','OUTETNO']
for c in cols: 
    df_drain_raw_data[c] = df_drain_raw_data[c].apply(uf.remove_punction)


df_drain_raw_data['Data Date_Drainage'] = df_drain_raw_data['Data Date_Drainage'].apply(uf.regulate_timestamp_format)

In [57]:

#question to be answered: what to do with meanless or null values.
#return null if DrainageWorksheet_SYSNO,DrainageWorksheet_INETNO,DrainageWorksheet_OUTETNO are null or empty

# df_drain_raw_data['Unique Culvert ID'] = (df_drain_raw_data['SYSNO'] + "_"
#                                           + df_drain_raw_data['INETNO'] + "_"
#                                          + df_drain_raw_data['OUTETNO'])

In [58]:
# df_drain_raw_data['SYSNO'].value_counts(dropna = False)

In [59]:
def calc_drain_unique_ID(df):
    if pd.isnull(df['SYSNO']) or pd.isnull(df['INETNO']) or pd.isnull(df['OUTETNO']):
        return None
    else:
        return (df['SYSNO'] + "_"+ df['INETNO'] + "_"+ df['OUTETNO'])
df_drain_raw_data['Unique Culvert ID'] = df_drain_raw_data.apply(calc_drain_unique_ID, axis = 1)

<a id='TMS_Worksheet'></a> 
## Raw Data TMS Worksheet

In [60]:
dict_TMS_rename = {
 'ID': 'AMT_ID',
 'Data Date':'Data Date_TMS'
                               }
df_tms_raw_data.rename(dict_TMS_rename, axis = 1, inplace = True)

df_tms_raw_data.name = 'df_tms_raw_data'

df_tms_raw_data.shape

df_tms_raw_data['Data Date_TMS'] = df_tms_raw_data['Data Date_TMS'].apply(uf.regulate_timestamp_format)

In [61]:
df_Minor_tms_raw_data.rename(dict_TMS_rename, axis = 1, inplace = True)

df_Minor_tms_raw_data['Data Date_TMS'] = df_Minor_tms_raw_data['Data Date_TMS'].apply(uf.regulate_timestamp_format)

In [62]:
df_HM_tms_raw_data.rename(dict_TMS_rename, axis = 1, inplace = True)

df_HM_tms_raw_data['Data Date_TMS'] = df_HM_tms_raw_data['Data Date_TMS'].apply(uf.regulate_timestamp_format)

In [63]:
df_tms_raw_data = pd.merge(df_tms_raw_data, df_SHOPP_raw_data[['AMT_ID','Section In Use']], how = 'left', 
                  left_on = ['AMT_ID'], 
                  right_on =  ['AMT_ID'])


df_tms_raw_data.rename(dict_TMS_rename, axis = 1, inplace = True)

In [64]:
df_Minor_tms_raw_data = pd.merge(df_Minor_tms_raw_data, df_Minor_raw_data[['AMT_ID','Section In Use']], how = 'left', 
                  left_on = ['AMT_ID'], 
                  right_on =  ['AMT_ID'])



In [65]:
df_HM_tms_raw_data = pd.merge(df_HM_tms_raw_data, df_HM_raw_data[['AMT_ID','Section In Use']], how = 'left', 
                  left_on = ['AMT_ID'], 
                  right_on =  ['AMT_ID'])



In [66]:
def Ck_HM_ActiveProject(df):
    try:
        if df['RTL'] == '0000': 
            return 'No'
        elif int(df['RTL'][-2:]) > 22: 
            return 'No'
        else:
            return 'Yes'
    except:
        return 'No'

In [67]:
df_HM_raw_data['Active Project?'] = df_HM_raw_data.apply(Ck_HM_ActiveProject, axis = 1)

In [68]:
df_Minor_ApprovedList['Active Project?'] = 'Yes'

df_Minor_raw_data = pd.merge(df_Minor_raw_data, df_Minor_ApprovedList[['EFIS','Active Project?']], how = 'left', 
                  left_on = ['EFIS'], 
                  right_on =  ['EFIS'])

df_Minor_raw_data['Active Project?'].fillna('No', inplace=True)

In [69]:
#Mark SHOPP active project
def mark_active_SHOPP_project(**kwargs):

    isPlaceholder = kwargs['Ten-Year Plan'] == 9999
    
    reservation_keywords = ['Major Damage - Emergency Opening',
    'Major Damage - Permanent Restoration',
    'Reactive Safety',
    'Relinquishment',
    'Safety - SI',]

    isReservationProject = kwargs['Activity Category'] in reservation_keywords
    
    isProgrammed = pd.notnull(kwargs['SHOPP Amendment Date'])
    
    if pd.isnull(kwargs['RTL In Use']):
        RTL = 0 #to be answered by mara
    else:
        RTL = int(kwargs['RTL In Use'][-2:])
    
    if isPlaceholder: 
        return 'No'
    elif RTL < 32: 
        return 'Yes'
    elif RTL < 27 and (not isProgrammed) and (not isReservationProject): 
        return 'No'
    else:
        return 'Yes'
    


In [70]:
df_SHOPP_raw_data['Active Project?'] = df_SHOPP_raw_data.apply( lambda df: mark_active_SHOPP_project(
**{'Ten-Year Plan': df['Ten-Year Plan RD'],
'Activity Category': df['Activity'],
'SHOPP Amendment Date': df['SHOPP Amendment Date'],
'RTL In Use': df['RTL'],}
), axis = 1
)

In [71]:
df_SHOPP_raw_data[df_SHOPP_raw_data['AMT_ID'] == 23439   ][['AMT_ID','Ten-Year Plan RD','Active Project?']]

,AMT_ID,Ten-Year Plan RD,Active Project?
5249,23439,9999,No


In [72]:
df_SHOPP_raw_data.drop(columns=['Cca Finish Date', 'Cca Percent Comp'],inplace=True , errors='ignore')

df_SHOPP_raw_data = pd.merge(df_SHOPP_raw_data, df_Project_Details, how = 'left', 
                  left_on = ['EFIS'], 
                  right_on = ['EFIS'])

df_SHOPP_raw_data['Cca Percent Comp'].fillna(0, inplace=True)
df_SHOPP_raw_data['Cca Finish Date'].fillna('', inplace=True)

In [73]:
# TMS_cols = ['TMSID', 'AMT_ID', 'Section', 'Program', 'District', 'TMS Structural or Technology', 'Asset Pre-Condition at RTL', 'Section_In_Use']

df_tms_all = df_tms_raw_data.append(df_Minor_tms_raw_data).append(df_HM_tms_raw_data)

df_tms_all['Same as section in use?'] = df_tms_all['Section'] ==df_tms_all['Section In Use']


In [74]:
#TMS worksheet in PPC section?

df_tms_in_PPC = df_tms_all[(df_tms_all['Section'] == 'PPC') & (df_tms_all['TMS Structural or Technology'].isin(['Technology & Structures', 'Technology']))][['AMT_ID']].groupby('AMT_ID').first().reset_index()

df_tms_in_PPC['TMS worksheet in PPC section?'] ='Yes'


df_SHOPP_raw_data.drop(['TMS worksheet in PPC section?'],
  axis='columns', inplace=True, errors='ignore')

df_SHOPP_raw_data = pd.merge(df_SHOPP_raw_data, df_tms_in_PPC, how = 'left', 
                  left_on = ['AMT_ID'], 
                  right_on = ['AMT_ID'])

df_SHOPP_raw_data['TMS worksheet in PPC section?'].fillna('No', inplace=True)

In [75]:
df_perf_raw_data = pd.merge(df_perf_raw_data, df_SHOPP_raw_data[['AMT_ID','Section']], how = 'inner', 
                  left_on = ['AMT_ID','Section'], 
                  right_on =  ['AMT_ID','Section'])

SHOPP_TMS_perf_Summary = df_perf_raw_data[df_perf_raw_data['Performance Objective'] == 'Transportation Management Systems'].groupby(['AMT_ID'])[['Assets in Good Cond', 
       'Assets in Poor Cond', 'New Assets Added',]].sum().reset_index()

dict_rename = {
     'Assets in Good Cond': 'AM Tool TMS Good',
    'Assets in Poor Cond': 'AM Tool TMS Poor',
    'New Assets Added': 'AM Tool TMS New',
                               }
SHOPP_TMS_perf_Summary.rename(dict_rename, axis = 1, inplace = True)

In [76]:
SHOPP_TMS_perf_Summary['Claims TMS technology Performance'] = 'Yes'

#column trimming
df_SHOPP_raw_data.drop(['Claims TMS technology Performance'],
  axis='columns', inplace=True, errors='ignore')

df_SHOPP_raw_data = pd.merge(df_SHOPP_raw_data, SHOPP_TMS_perf_Summary, how = 'left', 
                  left_on = ['AMT_ID'], 
                  right_on = ['AMT_ID'])

df_SHOPP_raw_data['Claims TMS technology Performance'].fillna('No', inplace=True)

In [77]:
df_SHOPP_raw_data['AM Tool RTL (Section in Use)'] = df_SHOPP_raw_data.apply(uf.calc_SIU_RTL, axis=1)

df_SHOPP_raw_data['AM Tool RTL (Section in Use)'].fillna('00', inplace = True)

In [78]:
#Last Year FY POR

# cal ='''
# FLOAT(Right([AM Tool RTL (Section in Use)],2))
# '''

df_SHOPP_raw_data['Last Year FY POR'] = df_SHOPP_raw_data['AM Tool RTL (Section in Use)'].str[-2:].astype(int)+2000

In [79]:
df_SHOPP_raw_data['Activity (group)'] = df_SHOPP_raw_data['Activity'].apply(uf.calc_activity_group)

In [80]:
def calc_include_5year_POR(df):
    if df['Ten-Year Plan RD'] == 9999:
        return  'No'
    elif(df['Last Year FY POR']>TARGET_FY and df['Last Year FY POR']<TARGET_FY + 6) :   
        if df['Activity (group)'] == 'Reservation' and pd.isnull(df['SHOPP Amendment Date']): 
            return  'Yes' 
        else: 
            return 'No' 
    elif (df['Last Year FY POR']>TARGET_FY + 5 and df['Last Year FY POR']<TARGET_FY + 11) : 
        if(df['Long Lead'] == "Y") and (df['Section'] == "PRG")  :
             return  'Yes'
        elif pd.isnull(df['SHOPP Amendment Date']): 
             return  'Yes' 
        else: 
            return 'No' 
    else: 
        return 'No'

df_SHOPP_raw_data['Include 5-year POR?'] = df_SHOPP_raw_data.apply(calc_include_5year_POR, axis = 1)

NameError: name 'TARGET_FY' is not defined

In [ ]:
col_active_projects = ['AMT_ID', 'EFIS', 'RTL', 'Active Project?']

df_active_projects = df_SHOPP_raw_data[col_active_projects].append(df_Minor_raw_data[col_active_projects]).append( df_HM_raw_data[col_active_projects])

df_tms_all = pd.merge(df_tms_all, df_active_projects, how = 'left', 
                  left_on = ['AMT_ID'], 
                  right_on = ['AMT_ID'])

In [ ]:
df_tms_all.drop(columns=['Cca Finish Date', 'Cca Percent Comp'],inplace=True , errors='ignore')

df_tms_all = pd.merge(df_tms_all, df_Project_Details[['EFIS','Cca Finish Date', 'Cca Percent Comp']], how = 'left', 
                  left_on = ['EFIS'], 
                  right_on = ['EFIS'])

df_tms_all['Cca Finish Date'].fillna("No CCA Date", inplace=True)
df_tms_all['Cca Percent Comp'].fillna(0, inplace=True)

In [ ]:
df_tms_all.drop(columns=['Include 5-year POR?'],inplace=True , errors='ignore')
df_tms_all = pd.merge(df_tms_all, df_SHOPP_raw_data[['AMT_ID', 'Section In Use', 'Include 5-year POR?']], how = 'left', 
                  left_on = ['AMT_ID', 'Section In Use'], 
                  right_on = ['AMT_ID', 'Section In Use'])

df_tms_all['Include 5-year POR?'].fillna("No", inplace=True)

In [ ]:
temp = df_tms_all[(df_tms_all['Active Project?'] == 'Yes') 
           & (df_tms_all['Same as section in use?'] == True) 
           & (df_tms_all['TMS Structural or Technology'] != 'Structures')
          & (df_tms_all['Cca Percent Comp'] == 0)].groupby('TMSID')['AMT_ID'].count().reset_index()

In [ ]:
temp['Repeated TMS?'] = temp['AMT_ID'].apply(lambda x: 'Yes' if x > 1 else 'No')

In [ ]:
# temp

In [ ]:
df_tms_all = pd.merge(df_tms_all, temp[['TMSID','Repeated TMS?' ]], how = 'left', 
                  left_on = ['TMSID'], 
                  right_on = ['TMSID'])

In [ ]:
df_tms_all = pd.merge(df_tms_all, df_TMS_LifeCycle, how = 'left', 
                  left_on = ['Asset Type'], 
                  right_on = ['Asset Type'])

df_tms_all['Repeated TMS?'].fillna("Not included", inplace=True)

In [ ]:
def ck_repeated_TMS(df):
    if df['TMS Structural or Technology'] == 'Structures':
        return 'Structures'
    elif df['Asset Pre-Condition at RTL'] == 'New':
        return 'New Asset'
    else:
        return df['Repeated TMS?']

In [ ]:
df_tms_all['Repeated TMS?'] = df_tms_all.apply(ck_repeated_TMS, axis = 1)

In [ ]:
df_tms_all['RTL2'] = df_tms_all['RTL'].str[-2:].astype(int)

In [ ]:
def calc_delta_RTL(s):
    RTLs = np.array(s.values)
    RTLs.sort()
    return min(np.diff(RTLs))
    
#     return RTLs[-1] - RTLs[-2]

In [ ]:
test_s = pd.Series([28,22,24,30])

calc_delta_RTL(test_s)

In [ ]:
temp = df_tms_all[df_tms_all['Repeated TMS?']=='Yes'].groupby(['TMSID']).agg({
    'RTL2': calc_delta_RTL,
    'Asset Life Cycle Limit': min
}).reset_index()

In [ ]:
dict_rename = {
    'RTL2': 'Delta RTL',
    }
temp= temp.rename(dict_rename, axis = 1)

In [ ]:
temp['Is Repeated TMS ID within Life Cycle?'] = temp.apply(lambda df: 'Yes' if df['Asset Life Cycle Limit'] > df['Delta RTL'] else 'No', axis = 1)

In [ ]:
# temp2 = pd.merge(temp[temp['Is Repeated TMS ID within Life Cycle?'] == 'Yes'], 
#          df_tms_all[['District', 'TMSID','Asset Type', 'Program', 'AMT_ID', 'Section','EA', 'EFIS','Include 5-year POR?','RTL']],
#          how = 'left', left_on=['TMSID'], right_on=['TMSID'])

# temp2

In [ ]:
# temp2[temp2['TMSID']=='1003-12FRMBQ']

In [ ]:
# df_tms_all[df_tms_all['TMSID']=='1003-12FRMBQ']

In [ ]:
df_duplicated_TMS = pd.merge(temp, 
         df_tms_all[['District', 'TMSID','Asset Type', 'Program', 'AMT_ID', 'Section','EA', 'EFIS','Include 5-year POR?','RTL','Active Project?']],
         how = 'left', left_on=['TMSID'], right_on=['TMSID']).sort_values('TMSID')

In [ ]:
# df_duplicated_TMS[df_duplicated_TMS['TMSID']=='1003-12FRMBQ']

In [ ]:
# df_duplicated_TMS[(df_duplicated_TMS['District'] == 12)
#                   &(df_duplicated_TMS['Is Repeated TMS ID within Life Cycle?'] == 'Yes')
#                  &(df_duplicated_TMS['Include 5-year POR?'] == 'Yes')]

In [ ]:
# df_duplicated_TMS[(df_duplicated_TMS['TMSID'] == '12CCTV164Via')
#                   ]

<a id='Export_Output'></a>

### Export Output

In [ ]:
out_cols = ['District', 'TMSID', 'Asset Type',
       'Program', 'AMT_ID', 'Section', 'EA', 'EFIS', 'Include 5-year POR?', 'RTL', 'Delta RTL', 'Asset Life Cycle Limit',
       'Is Repeated TMS ID within Life Cycle?',]

df_out = df_duplicated_TMS
hyper_name = 'Duplicate_TMS_Summary'
uf.export_hyper(df_out, hyper_name, LOG_FILE)

In [ ]:
df_out = df_tms_all
hyper_name = 'TMS_With_Duplicate_Check'
uf.export_hyper(df_out, hyper_name, LOG_FILE)

In [ ]:
df_out = df_SHOPP_raw_data

hyper_name = 'SHOPP_RawData_w_TMS_Info'
uf.export_hyper(df_out, hyper_name, LOG_FILE)


<a id='FinalCleanUp'></a>
## Final Clean Up

In [ ]:
#clean up tableau publishing log file

import os
import glob
# get a recursive list of file paths that matches pattern
fileList = glob.glob('./*.log')
# Iterate over the list of filepaths & remove each file.
for filePath in fileList:
    try:
        os.remove(filePath)
    except OSError:
        print("Error while deleting file")


In [ ]:
end_time =  time.time()
elapsed = end_time - start_time
print('time elapsed : {} seconds'.format(elapsed))

file_export_log = open(LOG_FILE, "a")  # append mode
file_export_log.write('#####time elapsed : {} seconds \n'.format(elapsed))
file_export_log.close()
